In [ ]:
import cv2 as cv
import glob
import pickle

from matplotlib import pyplot as plt
from os import path

%matplotlib inline

In [ ]:
print('Using OpenCV version', cv.__version__)

In [ ]:
with open('data/labels.pickle', 'rb') as handle:
  data = pickle.load(handle)

In [ ]:
# Get and normalize file paths.
img_paths = [path.normpath(p) for p in glob.iglob('data/WIDER/test/**/*.jpg', recursive=True)]

face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

print(len(img_paths))

cv_dict = {}
for img_path in img_paths:
    img = cv.imread(img_path)
    grayscale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    detected_faces = face_cascade.detectMultiScale(grayscale_img, 1.3, 5)
    
    img_name = path.basename(img_path)
    cv_dict[img_name] = len(detected_faces)

In [ ]:
#Test performance of OpenCV
hints = 0;
missed = 0;

for fidx, folder in enumerate(data['img_names']):
    for idx, img_name in enumerate(folder):
        target_detected_faces = len(data['face_rectangles'][fidx][idx])
        predicted_detected_faces = cv_dict[img_name]
        hints = hints + min(target_detected_faces, predicted_detected_faces)
        missed = missed + abs(target_detected_faces - predicted_detected_faces)        

In [ ]:
print(hints)
print(missed)